In [1]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from helpers import recursive_find, read_json, create_fedora_results_table

In [2]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("artifacts/results/extracted/fedora", "*.json"))
print('Found %s experiment library files.' % len(experiments))

Found 3168 experiment library files.


In [3]:
df = create_fedora_results_table(experiments)
display(df)

,a,b,original,changed,analysis,seconds,predictor,prediction
0,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,missing-previously-found-symbols,0.000026,symbols,True
1,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,missing-previously-found-exports,0.000015,symbols,True
2,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,abidiff,0.09467,libabigail,True
3,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,abi-compliance-tester,6.976217,abi-laboratory,True
4,fedora-libs-36,fedora-libs-37,first/usr/lib64/gconv/ANSI_X3.110.so,second/usr/lib64/gconv/ANSI_X3.110.so,missing-previously-found-symbols,0.000027,symbols,True
...,...,...,...,...,...,...,...,...
12667,fedora-libs-34,fedora-libs-36,first/usr/lib64/kde3/kimg_dds.so,second/usr/lib64/kde3/kimg_dds.so,abi-compliance-tester,13.929454,abi-laboratory,True
12668,fedora-libs-34,fedora-libs-35,first/usr/lib64/libkdeinit_kcmshell.so,second/usr/lib64/libkdeinit_kcmshell.so,missing-previously-found-symbols,0.000101,symbols,True
12669,fedora-libs-34,fedora-libs-35,first/usr/lib64/libkdeinit_kcmshell.so,second/usr/lib64/libkdeinit_kcmshell.so,missing-previously-found-exports,0.000022,symbols,True
12670,fedora-libs-34,fedora-libs-35,first/usr/lib64/libkdeinit_kcmshell.so,second/usr/lib64/libkdeinit_kcmshell.so,abidiff,0.510176,libabigail,True


In [4]:
# Symbols has two tests - we are using missing-previously-found-symbols
df = df[df["analysis"] != "missing-previously-found-exports"]

# Remove unknowns
# df = df[df["prediction"] != "Unknown"]

# disagree = df.groupby(["a","b", "original", "changed"])['prediction'].nunique() > 1
# disagree = pandas.DataFrame(disagree)
# Where all predictors agree
all_predictors_agree = pandas.DataFrame(df.groupby(['a', 'b', 'original', 'changed'])['prediction'].nunique() == 1).reset_index()
df_all_agree = df.merge(all_predictors_agree, on=['a', 'b', 'original', 'changed'], suffixes=["", "_all_agree"])
df_all_agree = df_all_agree[df_all_agree['prediction_all_agree']]
df_all_agree

,a,b,original,changed,analysis,seconds,predictor,prediction,prediction_all_agree
0,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,missing-previously-found-symbols,0.000026,symbols,True,True
1,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,abidiff,0.09467,libabigail,True,True
2,fedora-libs-35,fedora-libs-37,first/usr/lib64/dovecot/lib20_mail_log_plugin.so,second/usr/lib64/dovecot/lib20_mail_log_plugin.so,abi-compliance-tester,6.976217,abi-laboratory,True,True
3,fedora-libs-36,fedora-libs-37,first/usr/lib64/gconv/ANSI_X3.110.so,second/usr/lib64/gconv/ANSI_X3.110.so,missing-previously-found-symbols,0.000027,symbols,True,True
4,fedora-libs-36,fedora-libs-37,first/usr/lib64/gconv/ANSI_X3.110.so,second/usr/lib64/gconv/ANSI_X3.110.so,abidiff,0.018693,libabigail,True,True
...,...,...,...,...,...,...,...,...,...
9499,fedora-libs-34,fedora-libs-36,first/usr/lib64/kde3/kimg_dds.so,second/usr/lib64/kde3/kimg_dds.so,abidiff,0.289855,libabigail,True,True
9500,fedora-libs-34,fedora-libs-36,first/usr/lib64/kde3/kimg_dds.so,second/usr/lib64/kde3/kimg_dds.so,abi-compliance-tester,13.929454,abi-laboratory,True,True
9501,fedora-libs-34,fedora-libs-35,first/usr/lib64/libkdeinit_kcmshell.so,second/usr/lib64/libkdeinit_kcmshell.so,missing-previously-found-symbols,0.000101,symbols,True,True
9502,fedora-libs-34,fedora-libs-35,first/usr/lib64/libkdeinit_kcmshell.so,second/usr/lib64/libkdeinit_kcmshell.so,abidiff,0.510176,libabigail,True,True


In [5]:
# Convert to wide format where analysis and predictor are columns.
df_wide = pandas.pivot(df, columns=['analysis', 'predictor'], index=['a', 'b', 'original', 'changed'], values='prediction')
df_wide

analysis                                                                                                                            missing-previously-found-symbols  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                          Unknown   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib64/libquadmath.so.0.0.0                                       Unknown   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/libncurses.so.6.2                  second/usr/lib64/libncurses.so.6.2                                             True   
                              first/usr/lib64/libncursesw.so.6.2                 second/usr/lib64/libncursesw.so.6.2                                            True   
                              first/usr/lib64/libnettle.so.8.4                   second/usr/lib64/libnettle.so.8.4                                              True   
                              first/usr/lib64/libnghttp2.so.14.21.1              second/usr/lib64/libnghttp2.so.14.21.1                                         True   
                              first/usr/lib64/libnpth.so.0.1.2                   second/usr/lib64/libnpth.so.0.1.2                                              True   

analysis                                                                                                                               abidiff  \
predictor                                                                                                                           libabigail   
a              b              original                                           changed                                                         
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...      False   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...      False   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...      False   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...      False   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib64/libquadmath.so.0.0.0                   False   
...                                                                                                                                        ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/libncurses.so.6.2                  second/usr/lib64/libncurses.so.6.2                       True   
                              first/usr/lib64/libncursesw.so.6.2                 second/usr/lib64/libncursesw.so.6.2                      True   
                              first/usr/l

In [6]:
# Join analysis/predictor columns
df_wide = df_wide.reset_index()
df_wide.columns = ["_".join(names) for names in df_wide.columns]
df_wide.columns

Index(['a_', 'b_', 'original_', 'changed_',
       'missing-previously-found-symbols_symbols', 'abidiff_libabigail',
       'abi-compliance-tester_abi-laboratory'],
      dtype='object')

In [7]:
# Get predictions only
df_predictions = df_wide[['missing-previously-found-symbols_symbols',
                          'abidiff_libabigail',
                          'abi-compliance-tester_abi-laboratory']]
df_predictions

,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
0,Unknown,False,False
1,Unknown,False,False
2,Unknown,False,False
3,Unknown,False,False
4,Unknown,False,False
...,...,...,...
3163,True,True,True
3164,True,True,True
3165,True,True,True
3166,True,True,True


In [8]:
# Filter to where predictions are either "True" or "False" only.
num_predictors = 3
df_predictions = df_predictions[(df_predictions == True).sum(axis=1) + (df_predictions == False).sum(axis=1) == num_predictors]

# Look to see if we have True/False
for column in df_predictions.columns:
    print(column)
    print(df_predictions[column].unique())
    
df_predictions

missing-previously-found-symbols_symbols
[True False]
abidiff_libabigail
[True False]
abi-compliance-tester_abi-laboratory
[True False]


,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
6,True,True,True
9,True,True,True
10,True,True,True
11,True,True,True
12,True,True,True
...,...,...,...
3163,True,True,True
3164,True,True,True
3165,True,True,True
3166,True,True,True


In [9]:
# Correlations between predictors.
# The NaN means that the values don't vary (as we see above, the symbols check is always true)
(df_predictions == True).corr()

,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
missing-previously-found-symbols_symbols,1.000000,0.445480,0.291874
abidiff_libabigail,0.445480,1.000000,0.521177
abi-compliance-tester_abi-laboratory,0.291874,0.521177,1.000000
